Imports

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from unbiasai.config import DATA_DIR
from unbiasai.utils import generate_embedding
from unbiasai.utils import get_embedding
#from deepseek import Embeddings

Load data

In [2]:
file_path = DATA_DIR / 'dataset_retrieval_recency.csv'
df = pd.read_csv(file_path)

Create subset of data for test

In [3]:
df_30 = df.head(30)

Load API key

In [4]:
load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")
model_name = "deepseek"

#api_key = os.getenv("COHERE_API_KEY")
#model_name = "cohere"

api_key = os.getenv("CLAUDE_API_KEY")
model_name = "claude"

In [5]:
#initialize llm
from unbiasai.utils import initialize_llm
llm = initialize_llm(model_name, api_key)

LLM initialized correctly: claude, llm: model='claude-3-7-sonnet-latest' anthropic_api_url='https://api.anthropic.com' anthropic_api_key=SecretStr('**********') model_kwargs={}


Generate embedding

In [6]:
df_30['embedding'] = df['content'].apply(generate_embedding, args=[api_key, model])

NameError: name 'model' is not defined

Insert documents in supabase table, IMPORTANT: change function so supabase table name can be changed

In [ ]:
insert_documents(df)

Connect to Supabase

In [ ]:
#Connect to Supabase
 ### Supabase Credentials
SUPABASE_URL = "https://rrjbrtbsvdoxndchvchq.supabase.co"
SUPABASE_KEY = supabase_key


# Create Supabase client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

Retrieve Documents for Each Query Across All Models

In [ ]:
# Initialize a list to collect data

models = ["gpt",  "claude", "mistral", "cohere", "deepseek"]
# retrieval_results structure: { model_name: { query: [list of document results] } }
retrieval_results = {model: {} for model in models}

for model in models:
    for query in test_queries:
        # Set re_rank=True if you wish to re-rank documents using the LLM.
        retrieval_results[model][query] = retrieve(query, model, k=4, re_rank=True)
print("Retrieval complete for all models and queries.")